In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch.utils.data as Data
import matplotlib.pyplot as plt
import seaborn as sns
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
housedata=fetch_california_housing()
train_x,test_x,train_y,test_y=train_test_split(housedata.data,housedata.target,test_size=0.3,random_state=42)
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
train_x=scale.fit_transform(train_x)
test_x=scale.fit_transform(test_x)
train_x.shape,train_y.shape,test_x.shape,test_y.shape,min(test_y),max(train_y)

((14448, 8), (14448,), (6192, 8), (6192,), 0.14999, 5.00001)

In [2]:
train_x=np.array(train_x)
train_y=np.array(train_y)
test_x=np.array(test_x)
test_y=np.array(test_y)

In [3]:
# Split dataset
def split_list_n_list(origin_list, n):
    if len(origin_list) % n == 0:
        cnt = len(origin_list) // n
    else:
        cnt = len(origin_list) // n + 1
 
    for i in range(0, n):
        yield origin_list[i*cnt:(i+1)*cnt]

In [4]:
#preparing teacher's datasets
n_teachers=5

teacher_x,teacher_y = [],[]
teacher_datasets = []
teacher_data_loader = []



teacher_x_loder = split_list_n_list(train_x,n_teachers)
teacher_y_loder = split_list_n_list(train_y,n_teachers)

teacher_x.extend(iter(teacher_x_loder))
teacher_y.extend(iter(teacher_y_loder))

for i in range(n_teachers):
    print(i,teacher_x[i].shape,teacher_y[i].shape)


from torch.utils.data import TensorDataset, DataLoader
teacher_datasets.extend(TensorDataset(torch.tensor(teacher_x[u],device=device,dtype=torch.float),
                                       torch.tensor(teacher_y[u],device=device,dtype=torch.float))
                        for u in  range(n_teachers))

teacher_data_loader.extend(DataLoader(teacher_datasets[i],batch_size=128,shuffle=False)
                           for i in range(n_teachers))


student_x,student_y = [] ,[]
student_datasets= []
 

student_x_loder =  split_list_n_list(test_x,2)
student_y_loder =  split_list_n_list(test_y,2)
student_x.extend(iter(student_x_loder))
student_y.extend(iter(student_y_loder))

for i in range(2):
    print(i,student_x[i].shape,student_y[i].shape)

student_datasets.extend(
                        TensorDataset(torch.tensor(student_x[u],device=device,dtype=torch.float),
                                      torch.tensor(student_y[u],device=device,dtype=torch.float))
                        for u in range(2)
)

student_train_loader = DataLoader(student_datasets[0], batch_size=len(student_datasets[0]),shuffle=True)
student_test_loader = DataLoader(student_datasets[1], batch_size=len(student_datasets[1]),shuffle=True)

0 (2890, 8) (2890,)
1 (2890, 8) (2890,)
2 (2890, 8) (2890,)
3 (2890, 8) (2890,)
4 (2888, 8) (2888,)
0 (3096, 8) (3096,)
1 (3096, 8) (3096,)


In [5]:
from random import shuffle
from torch.utils.data import TensorDataset, DataLoader
 
from opacus.utils.uniform_sampler import UniformWithReplacementSampler

train_x=np.array(train_x)
train_y=np.array(train_y)
test_x=np.array(test_x)
test_y=np.array(test_y)


train_dataset=TensorDataset(torch.tensor(train_x,dtype=torch.float),torch.tensor(train_y,dtype=torch.float))
train_loader=DataLoader(train_dataset,num_workers=1, batch_size=128,shuffle=True)

test_dataset=TensorDataset(torch.tensor(test_x,dtype=torch.float)
                    ,torch.tensor(test_y,dtype=torch.float))
test_loader=DataLoader(train_dataset,num_workers=1, batch_size=128,shuffle=True)
len(train_dataset)

14448

In [6]:
# neural network model mlp
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden1 = nn.Sequential(nn.Linear(in_features=8, out_features=256, bias=True),
                                  nn.Dropout(0.7),
                                  nn.ReLU())
        
        self.hidden2 = nn.Sequential(nn.Linear(in_features=256, out_features=64, bias=True),
                                  nn.Dropout(0.7),
                                  nn.ReLU())
        
        self.hidden3 = nn.Sequential(nn.Linear(in_features=64, out_features=16, bias=True),
                                  nn.Dropout(0.5),
                                  nn.ReLU()
                                  )
        
        self.predict = nn.Sequential(nn.Linear(in_features=16, out_features=1, bias=True),
                                        nn.ReLU())
        
    def forward(self, x):
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        output = self.predict(x)
        return output[:, 0]




model=MLP().to(device)
teachers_models = []
import torch.nn as nn
import torch.optim as optim
for i in range(n_teachers):
    net = model
    teachers_models.append(net)
 
teacher_optimizers = [torch.optim.SGD(teachers_models[i].parameters(), lr=1e-3) for i in range(n_teachers)]
#creating teachers folders
def  mkdir_if_missing(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

for i in  range(n_teachers):
    mkdir_if_missing(f'/home/ysy/ysy/Fed-ReKD-dirs/MLP_DP_SGD_california/teacher{i}')


In [7]:
 
#training configs
num_epochs=500  #10*3000epoch
batch_size =128
lr =1e-4

In [8]:
#training teacher models
loss_func=nn.MSELoss()
train_loss_all=[]
for tea_num in range(n_teachers):
    print(f'start training number {tea_num} techer!')
    minloss =float ('inf')
    
    for epoch in range(num_epochs):
        train_loss = 0
        train_num=0
        teachers_models[tea_num].train()
        for b_x,b_y in teacher_data_loader[tea_num]:
            teacher_optimizers[tea_num].zero_grad()
            output=teachers_models[tea_num].forward(b_x.to(device))
            output=output.squeeze(-1)
            loss=loss_func(output,b_y.to(device))
            loss.backward()
            teacher_optimizers[tea_num].step()
            if loss< minloss:
                minloss = loss
                if os.path.exists(f'/home/ysy/ysy/Fed-ReKD-dirs/MLP_DP_SGD_california/teacher{tea_num}/best.pth'):
                    os.remove(f'/home/ysy/ysy/Fed-ReKD-dirs/MLP_DP_SGD_california/teacher{tea_num}/best.pth')
                torch.save(teachers_models[tea_num], f'/home/ysy/ysy/Fed-ReKD-dirs/MLP_DP_SGD_california/teacher{tea_num}/best.pth')
        if epoch%1== 0:
            print(f'Number:{tea_num}','epoch: {}/{} \t'.format(epoch+1,num_epochs),f'loss_tea:{loss}')
        train_loss_all.append(loss)
    print(f'finished training number {tea_num} techer!')

start training number 0 techer!
Number:0 epoch: 1/500 	 loss_tea:6.847187042236328
Number:0 epoch: 2/500 	 loss_tea:6.477248191833496
Number:0 epoch: 3/500 	 loss_tea:5.93139123916626
Number:0 epoch: 4/500 	 loss_tea:5.2429327964782715
Number:0 epoch: 5/500 	 loss_tea:4.205296993255615
Number:0 epoch: 6/500 	 loss_tea:3.90695858001709
Number:0 epoch: 7/500 	 loss_tea:3.4008607864379883
Number:0 epoch: 8/500 	 loss_tea:3.0016846656799316
Number:0 epoch: 9/500 	 loss_tea:2.8424510955810547
Number:0 epoch: 10/500 	 loss_tea:2.3615975379943848
Number:0 epoch: 11/500 	 loss_tea:2.7971115112304688
Number:0 epoch: 12/500 	 loss_tea:2.7604427337646484
Number:0 epoch: 13/500 	 loss_tea:2.3339743614196777
Number:0 epoch: 14/500 	 loss_tea:2.3846797943115234
Number:0 epoch: 15/500 	 loss_tea:2.0464046001434326
Number:0 epoch: 16/500 	 loss_tea:2.5243732929229736
Number:0 epoch: 17/500 	 loss_tea:2.4314510822296143
Number:0 epoch: 18/500 	 loss_tea:2.223036289215088
Number:0 epoch: 19/500 	 loss_t